**"" Program Name : Sig Norm**  
**"" Programmer : Jeong Seonil**  
**"" Virsion No. : 21.03.31**  
**"" Revision Date : 21.03.31**  
**"" Function :**

    해당 프로그램은 Frame Matching Rate 코드를 거쳐 생성된 동작 프레임의 파일을 읽어와서 각각의 프레임을 정규화한 후에 해당 폴더 밑의 Normalization 폴더에 저장하는 프로그램이다. 
    정규화 방법은 시간은 1000샘플에 맞추고 크기는 0을 최소로 1을 최대로 하여 0~1 사이로 정하는 정규화 방법을 사용한다.

In [16]:
import time
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as df
import seaborn as sns
import threading
import glob
import sys
import os
from scipy import signal
import pylab as pl
from enum import Enum
from IPython import display
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler

def all_subdirs_of(b='.'):
    result = []
    for d in os.listdir(b):
        bd = os.path.join(b, d)
        if os.path.isdir(bd): result.append(bd)
    return result

In [17]:
def Init():
    root_path = '../../Data/Motion Frame Data'
    date_list = []
    motion_list = []
    human_list = []
    motion_number = 0
    human_number = 0

    #files = os.listdir(root_path)
    #for i in files :
    #    if os.path.isdir(root_path + "/"+i):
    #        date_list.append(i)
    date_list = os.listdir(root_path)

    number = int(input("리스트에서 실험하고 싶은 데이터의 날짜를 선택하세요.(번호 1~{}사이)\n {} >> ".format(len(date_list),date_list)))
    global date_path
    date_path = root_path + "/"+ date_list[number-1]
    print('')

    name_flag = input('실험동작과 실험자를 입력하시겠습니까? 입력하지 않으면 전부 선택합니다. \n 입력하시는 경우는 Y를 입력하세요(Y/N) >> ')
    print('')

    if name_flag == 'Y':
        temp = os.listdir(date_path)
        print(temp)
        motion_number = int(input("실험 동작의 갯수를 입력하세요 >> "))
        print('')
        for i in range(motion_number):
            motion = int(input('위의 리스트에서 동작의 번호를 입력하세요. (번호 1~{}사이) >> '.format(len(temp))))
            print('')
            motion_list.append(temp[motion-1])
            print('선택된 동작 {}'.format(motion_list))
        temp = os.listdir(date_path + "/"+ motion_list[0])
        print(temp)
        human_number = int(input("실험자의 수를 입력하세요 >> "))
        print('')
        for i in range(human_number):
            human = int(input('위의 리스트에서 실험자의 번호를 입력하세요. (번호 1~{}사이) >> '.format(len(temp))))
            print('')
            human_list.append(temp[human-1])
            print('선택된 실험자 {}'.format(human_list))
    elif name_flag == 'N':
        motion_list = os.listdir(date_path)
        human_list = os.listdir(date_path + "/"+ motion_list[0])

    return motion_list,human_list

In [12]:
mod = sys.modules[__name__]

Day = datetime.today().strftime("%Y.%m.%d %H%M%S")
Initial_threshold = 0.022
interval_period = 1.5

motion_list, human_list = Init()

for motion in motion_list:
    for human in human_list:
        min_max_scaler = MinMaxScaler()
        
        
        human_path = date_path + '/' + motion + '/' + human
        A_path = human_path + '/Frame/' + 'A/*.csv'
        B_path = human_path + '/Frame/' + 'B/*.csv'

        path = '../../Data/Normalization Data/' + Day + '/' + motion + '/' + human + '/'
        if not os.path.isdir(path + 'A'):
            os.makedirs(path + 'A')
            os.makedirs(path + 'B')
            os.makedirs(path + 'A-B')

        A_Frame = glob.glob(A_path)
        B_Frame = glob.glob(B_path)

        File_number = len(A_Frame)

        for i in range(File_number):
        
            A = np.array(pd.read_csv(A_Frame[i]))
            B = np.array(pd.read_csv(B_Frame[i]))
            A_B =  A-B
            temp = len(A)%100
            len_compare = len(A) - temp
            
            Ori_A = A
            Ori_B = B
            Ori_AB = A_B
            
            Nor_A = []
            Nor_B = []
            Nor_AB = []

            if(len_compare > 1000):  #프레임 길이 센서 A, B 모두 1000을 넘을 때 
                div_len = len_compare/1000

                for k in range(1000):
                    Nor_A.append(Ori_A[int(k*div_len)])
                    Nor_B.append(Ori_B[int(k*div_len)])
                    Nor_AB.append(Ori_AB[int(k*div_len)])

            else:
                div_len = 1000/len_compare

                for k in range(1000):
                    Nor_A.append(Ori_A[int(k*div_len)])
                    Nor_B.append(Ori_B[int(k*div_len)])
                    Nor_AB.append(Ori_AB[int(k*div_len)])

                Nor_A = df(Nor_A)
                Nor_A = Nor_A.interpolate(method='values')
                Nor_B = df(Nor_B)
                Nor_B = Nor_B.interpolate(method='values')
                Nor_AB = df(Nor_AB)
                Nor_AB = Nor_AB.interpolate(method='values')
                
            Nor_A = df(Nor_A)
            Nor_B = df(Nor_B)
            Nor_AB = df(Nor_AB)
            Nor_A = min_max_scaler.fit_transform(Nor_A)
            Nor_B = min_max_scaler.fit_transform(Nor_B)
            Nor_AB = min_max_scaler.fit_transform(Nor_AB)
            
            (df(Nor_A)).to_csv(path + 'A/A_frame_{}.csv'.format(i),header=False,index=False)
            (df(Nor_B)).to_csv(path + 'B/B_frame_{}.csv'.format(i),header=False,index=False)
            (df(Nor_AB)).to_csv(path + 'A-B/AB_frame_{}.csv'.format(i),header=False,index=False)

리스트에서 실험하고 싶은 데이터의 날짜를 선택하세요.(번호 1~2사이)
 ['2021.04.01 200429', '2021.04.09 163032'] >> 2

실험동작과 실험자를 입력하시겠습니까? 입력하지 않으면 전부 선택합니다. 
 입력하시는 경우는 Y를 입력하세요(Y/N) >> Y

['Mo type 1. LR', 'Mo type 2. RL', 'Mo type 3. CW']
실험 동작의 갯수를 입력하세요 >> 1

위의 리스트에서 동작의 번호를 입력하세요. (번호 1~3사이) >> 1

선택된 동작 ['Mo type 1. LR']
['CJW', 'JSI', 'KB', 'KJY', 'KME', 'PJS']
실험자의 수를 입력하세요 >> 1

위의 리스트에서 실험자의 번호를 입력하세요. (번호 1~6사이) >> 2

선택된 실험자 ['JSI']
